# Collecting NBA Game Data and Extracting Features for Prediction Model

In [2]:
import csv
import numpy as np
import pandas as pd

## 1. Preparing the Data

I will try to build a linear regression model that can predict the margin of victory for NBA games. Due to time constraints, I will limit my features to those that can be extracted from game scores such as team record, points per game, points allowed per game, rest days, etc. To do this, I first have to scrape game scores data from basketball-reference.com

### 1.1 Scraping game scores from the web

In [3]:
import requests
resp = requests.get(
    "https://www.basketball-reference.com/leagues/NBA_2017_games.html")
resp

<Response [200]>

In [4]:
resp.content[0:500]

b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/201801291" />\n\n<!-- no:cookie fast load the css.           -->\n<link rel="preconnect" h'

In [5]:
encoding = resp.headers['Content-Type'].split("=")[-1]
encoding

'utf-8'

In [6]:
resp.content.decode(encoding)[0:500]

'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/201801291" />\n\n<!-- no:cookie fast load the css.           -->\n<link rel="preconnect" h'

In [7]:
from bs4 import BeautifulSoup
october = pd.read_html(resp.content, encoding=encoding)[0]
october.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,,.1,Attend.,Notes
0,"Tue, Oct 25, 2016",7:30 pm,New York Knicks,88,Cleveland Cavaliers,117,Box Score,NaN,20562,NaN
1,"Tue, Oct 25, 2016",10:30 pm,San Antonio Spurs,129,Golden State Warriors,100,Box Score,NaN,19596,NaN
2,"Tue, Oct 25, 2016",10:00 pm,Utah Jazz,104,Portland Trail Blazers,113,Box Score,NaN,19446,NaN
3,"Wed, Oct 26, 2016",7:30 pm,Brooklyn Nets,117,Boston Celtics,122,Box Score,NaN,18624,NaN
4,"Wed, Oct 26, 2016",7:00 pm,Dallas Mavericks,121,Indiana Pacers,130,Box Score,OT,17923,NaN


Let's make a helper function to clean tables of this format since we'll need to import a lot of them.

In [8]:
def clean_bbref_scores(df):
    df = df[['Date', 'Visitor/Neutral','PTS','Home/Neutral','PTS.1']]
    df.columns = ['date', 'away team', 'away pts', 'home team','home pts']
    
    return df

In [9]:
october = clean_bbref_scores(october)
october.head()

,date,away team,away pts,home team,home pts
0,"Tue, Oct 25, 2016",New York Knicks,88,Cleveland Cavaliers,117
1,"Tue, Oct 25, 2016",San Antonio Spurs,129,Golden State Warriors,100
2,"Tue, Oct 25, 2016",Utah Jazz,104,Portland Trail Blazers,113
3,"Wed, Oct 26, 2016",Brooklyn Nets,117,Boston Celtics,122
4,"Wed, Oct 26, 2016",Dallas Mavericks,121,Indiana Pacers,130


Now let's scrape the data from all the months in the same manner.

In [10]:
months = ['october', 'november','december','january','february','march','april']
dfs_2017 = []

for month in months:
    address = "https://www.basketball-reference.com/leagues/NBA_2017_games-"+month+".html"
    resp = requests.get(address)
    df = pd.read_html(resp.content, encoding=encoding)[0]
    df = clean_bbref_scores(df)
    dfs_2017.append(df)

In [11]:
scores_2017 = pd.concat(dfs_2017).reset_index(drop=True)
scores_2017.shape

(1276, 5)

In [12]:
scores_2017.tail()

,date,away team,away pts,home team,home pts
1271,"Fri, Apr 28, 2017",Washington Wizards,115.0,Atlanta Hawks,99.0
1272,"Fri, Apr 28, 2017",Boston Celtics,105.0,Chicago Bulls,83.0
1273,"Fri, Apr 28, 2017",Los Angeles Clippers,98.0,Utah Jazz,93.0
1274,"Sun, Apr 30, 2017",Washington Wizards,111.0,Boston Celtics,123.0
1275,"Sun, Apr 30, 2017",Utah Jazz,104.0,Los Angeles Clippers,91.0


Looks good! Still, there is one small problem. The playoffs began on April 15th of that year, and we only want to consider regular season games. Therefore, we actually have to trim our data set to only include games before the 15th. This would be easier if we could convert the "date" column into datetime objects, but there is a weirdly formatted row with "Playoffs" as the date to demarcate the start of the playoffs. So we'll have to do it by hand. 

In [13]:
scores_2017[scores_2017['date']=='Playoffs']

,date,away team,away pts,home team,home pts
1230,Playoffs,NaN,NaN,NaN,NaN


In [14]:
scores_2017 = scores_2017.iloc[:1230,:]
scores_2017.tail()

,date,away team,away pts,home team,home pts
1225,"Wed, Apr 12, 2017",Philadelphia 76ers,113.0,New York Knicks,114.0
1226,"Wed, Apr 12, 2017",Denver Nuggets,111.0,Oklahoma City Thunder,105.0
1227,"Wed, Apr 12, 2017",Detroit Pistons,109.0,Orlando Magic,113.0
1228,"Wed, Apr 12, 2017",New Orleans Pelicans,103.0,Portland Trail Blazers,100.0
1229,"Wed, Apr 12, 2017",San Antonio Spurs,97.0,Utah Jazz,101.0


All set. Now I'll scrape the 2018 regular season scores.

In [15]:
months = ['october', 'november','december','january','february','march']
dfs_2018 = []

for month in months:
    address = "https://www.basketball-reference.com/leagues/NBA_2018_games-"+month+".html"
    resp = requests.get(address)
    df = pd.read_html(resp.content, encoding=encoding)[0]
    df = clean_bbref_scores(df)
    dfs_2018.append(df)

scores_2018 = pd.concat(dfs_2018).reset_index(drop=True)
scores_2018.shape

(1142, 5)

In [16]:
# Getting rid of missing games from March

nan_bools = scores_2018['away pts'].isna().tolist()
nan_bools = [val == False for val in nan_bools]
scores_2018 = scores_2018.iloc[nan_bools,:]
scores_2018.shape

(1048, 5)

In [17]:
scores_2018.tail()

,date,away team,away pts,home team,home pts
1043,"Sat, Mar 17, 2018",Indiana Pacers,102.0,Washington Wizards,109.0
1044,"Sun, Mar 18, 2018",Portland Trail Blazers,122.0,Los Angeles Clippers,109.0
1045,"Sun, Mar 18, 2018",Houston Rockets,129.0,Minnesota Timberwolves,120.0
1046,"Sun, Mar 18, 2018",Boston Celtics,89.0,New Orleans Pelicans,108.0
1047,"Sun, Mar 18, 2018",Oklahoma City Thunder,132.0,Toronto Raptors,125.0


### 1.2 Cleaning the Data and storing as a csv

Finally, lets merge both tables into one big scores table and clean the data a bit. We will also add a "game_id" column to help us keep track of games later if we end up re_indexing the data.

In [18]:
scores = pd.concat([scores_2017,scores_2018]).reset_index(drop=True)
scores.shape

(2278, 5)

In [19]:
scores['date'] = pd.to_datetime(scores['date'])
type(scores['date'][0])

pandas._libs.tslib.Timestamp

In [20]:
teams_dict = {'Atlanta Hawks':'ATL', 'Boston Celtics':'BOS', 'Brooklyn Nets':'BKN',
             'Charlotte Hornets':'CHA', 'Chicago Bulls':'CHI', 'Cleveland Cavaliers':'CLE',
             'Dallas Mavericks':'DAL', 'Denver Nuggets':'DEN', 'Detroit Pistons':'DET',
             'Golden State Warriors':'GSW', 'Houston Rockets':'HOU', 'Indiana Pacers':'IND',
             'Los Angeles Clippers':'LAC','Los Angeles Lakers':'LAL','Memphis Grizzlies':'MEM',
             'Miami Heat':'MIA','Milwaukee Bucks':'MIL', 'Minnesota Timberwolves':'MIN',
             'New Orleans Pelicans':'NOP', 'New York Knicks':'NYK', 'Oklahoma City Thunder':'OKC',
             'Orlando Magic':'ORL','Philadelphia 76ers':'PHI','Phoenix Suns':'PHX',
             'Portland Trail Blazers':'POR','Sacramento Kings':'SAC','San Antonio Spurs':'SAS',
             'Toronto Raptors':'TOR','Utah Jazz':'UTA','Washington Wizards':'WAS'}

In [21]:
away_tms = scores['away team'].tolist()
home_tms = scores['home team'].tolist()

away_short = [teams_dict[away_tm] for away_tm in away_tms]
home_short = [teams_dict[home_tm] for home_tm in home_tms]

scores['away team'] = away_short
scores['home team'] = home_short

nums = range(len(scores))

scores['game_id'] = nums

scores.head(3)

,date,away team,away pts,home team,home pts,game_id
0,2016-10-25,NYK,88.0,CLE,117.0,0
1,2016-10-25,SAS,129.0,GSW,100.0,1
2,2016-10-25,UTA,104.0,POR,113.0,2


In [22]:
scores = scores[['game_id','date','home team', 'home pts', 'away team', 'away pts']]
scores.head(20)

,game_id,date,home team,home pts,away team,away pts
0,0,2016-10-25,CLE,117.0,NYK,88.0
1,1,2016-10-25,GSW,100.0,SAS,129.0
2,2,2016-10-25,POR,113.0,UTA,104.0
3,3,2016-10-26,BOS,122.0,BKN,117.0
4,4,2016-10-26,IND,130.0,DAL,121.0
5,5,2016-10-26,LAL,120.0,HOU,114.0
6,6,2016-10-26,MEM,102.0,MIN,98.0
7,7,2016-10-26,MIL,96.0,CHA,107.0
8,8,2016-10-26,NOP,102.0,DEN,107.0
9,9,2016-10-26,ORL,96.0,MIA,108.0


## 2. Feature Extraction

As I said before, I want to extract features from this data that might be useful at predicting game outcomes such as win %, pts per game, etc. However there is a problem. At the beginning of our dataset, the first few games of the 2016-17 season, some of these metrics could be misleading.

For example, if you look at the table above, the Indiana Pacers won their first game against the Dallas Mavericks. However, two days later they lost their second game against the Brooklyn Nets, who had in turn lost *their* first game against the Celtics. To our model, this would look like a major upset, a team with a 0% win percentage defeating a team with a 100% win percentage. However, because of the small sample size, this isn't really reflective of reality.

To resolve this, we will not consider each team's first 41 games of the 2016-2017 season in our analysis, and only use the remaining data to train and test our model. So, say instead of calculating "win %", we'll calculate "win% in last 41 games" for each record after each team's first 41 games. For the start of 2018 season, we'll use the teams winning percentage from the last 41 games of the previous seasons. Obviously this introduces some error as there was a lot of roster turnover last offseason, but we'll have to live with that for the sake of keeping our model simple.

### 2.1 Winning % last 41 games, last 8 games

We begin by computing each teams win % in their last 41 games. Because "streaks" can be indicative of how a team is doing, I will also compute the win % in the last 8 games.

In [23]:
# Make column recording the winner of the game

winners = []

for i in range(len(scores)):
    away_diff = scores['away pts'][i] - scores['home pts'][i]
    if away_diff > 0:
        winners.append(scores['away team'][i])
    else:
        winners.append(scores['home team'][i])

scores['winner'] = winners

scores.head()

,game_id,date,home team,home pts,away team,away pts,winner
0,0,2016-10-25,CLE,117.0,NYK,88.0,CLE
1,1,2016-10-25,GSW,100.0,SAS,129.0,SAS
2,2,2016-10-25,POR,113.0,UTA,104.0,POR
3,3,2016-10-26,BOS,122.0,BKN,117.0,BOS
4,4,2016-10-26,IND,130.0,DAL,121.0,IND


We now create a dataframe for each teams games, and store them in a dictionary.

In [24]:
teams = np.unique(scores['home team'])
team_dfs = dict.fromkeys(teams)  

for team in teams:
    df = scores.loc[(scores['home team']==team) | (scores['away team']==team), :]
    team_dfs[team] = df

In [25]:
team_dfs['LAL'].head()

,game_id,date,home team,home pts,away team,away pts,winner
5,5,2016-10-26,LAL,120.0,HOU,114.0,LAL
24,24,2016-10-28,UTA,96.0,LAL,89.0,UTA
39,39,2016-10-30,OKC,113.0,LAL,96.0,OKC
47,47,2016-11-01,IND,115.0,LAL,108.0,IND
54,54,2016-11-02,ATL,116.0,LAL,123.0,LAL


Now to each team's dataframe we add a column "team_won" which is 1 if the team won and 0 if the opponent won.

In [26]:
for team in teams:
    team_won = []
    tm_df = team_dfs[team]
    for i in range(len(tm_df)):
        if tm_df['winner'].iloc[i] == team:
            team_won.append(1)
        else:
            team_won.append(0)
    tm_df['team_won'] = team_won

/Users/Edgar/miniconda3/envs/data100/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
team_dfs['LAL'].head(10)

,game_id,date,home team,home pts,away team,away pts,winner,team_won
5,5,2016-10-26,LAL,120.0,HOU,114.0,LAL,1
24,24,2016-10-28,UTA,96.0,LAL,89.0,UTA,0
39,39,2016-10-30,OKC,113.0,LAL,96.0,OKC,0
47,47,2016-11-01,IND,115.0,LAL,108.0,IND,0
54,54,2016-11-02,ATL,116.0,LAL,123.0,LAL,1
72,72,2016-11-04,LAL,117.0,GSW,97.0,LAL,1
88,88,2016-11-06,LAL,119.0,PHX,108.0,LAL,1
101,101,2016-11-08,LAL,97.0,DAL,109.0,DAL,0
119,119,2016-11-10,SAC,91.0,LAL,101.0,LAL,1
136,136,2016-11-12,NOP,99.0,LAL,126.0,LAL,1


Next compute the rolling 41 game win % for each team in its dataframe.

In [28]:
for team in teams:
    team_df = team_dfs[team][['team_won']]
    roll_mean = team_df.rolling(window=41,win_type='boxcar').mean()
    team_dfs[team]['41 game win%'] = roll_mean

/Users/Edgar/miniconda3/envs/data100/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [29]:
team_dfs['LAL'].iloc[38:43,:]

,game_id,date,home team,home pts,away team,away pts,winner,team_won,41 game win%
540,540,2017-01-05,POR,118.0,LAL,109.0,POR,0,NaN
545,545,2017-01-06,LAL,127.0,MIA,100.0,LAL,1,NaN
559,559,2017-01-08,LAL,111.0,ORL,95.0,LAL,1,0.365854
572,572,2017-01-10,LAL,87.0,POR,108.0,POR,0,0.341463
589,589,2017-01-12,SAS,134.0,LAL,94.0,SAS,0,0.341463


And do the same for 8 game win%.

In [30]:
for team in teams:
    team_df = team_dfs[team][['team_won']]
    roll_mean = team_df.rolling(window=8,win_type='boxcar').mean()
    team_dfs[team]['8 game win%'] = roll_mean

/Users/Edgar/miniconda3/envs/data100/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
team_dfs['LAL'].iloc[38:43,:]

,game_id,date,home team,home pts,away team,away pts,winner,team_won,41 game win%,8 game win%
540,540,2017-01-05,POR,118.0,LAL,109.0,POR,0,NaN,0.250
545,545,2017-01-06,LAL,127.0,MIA,100.0,LAL,1,NaN,0.375
559,559,2017-01-08,LAL,111.0,ORL,95.0,LAL,1,0.365854,0.500
572,572,2017-01-10,LAL,87.0,POR,108.0,POR,0,0.341463,0.375
589,589,2017-01-12,SAS,134.0,LAL,94.0,SAS,0,0.341463,0.375


Finally, we can add these columns back into our original scores dataframe.

In [32]:
away_41 = []
for i in range(len(scores)):
    away_tm = scores['away team'][i]
    team_df = team_dfs[away_tm]
    win_41 = team_df.loc[i]['41 game win%']
    away_41.append(win_41)
scores['away team 41 game win%'] = away_41

In [33]:
scores.tail(3)

,game_id,date,home team,home pts,away team,away pts,winner,away team 41 game win%
2275,2275,2018-03-18,MIN,120.0,HOU,129.0,HOU,0.756098
2276,2276,2018-03-18,NOP,108.0,BOS,89.0,NOP,0.585366
2277,2277,2018-03-18,TOR,125.0,OKC,132.0,OKC,0.658537


Let's make sure everything worked well by verifying Miami's 41 game win% is actually 0.536585 in their last game.

In [34]:
team_dfs['MIA'].tail(1)

,game_id,date,home team,home pts,away team,away pts,winner,team_won,41 game win%,8 game win%
2258,2258,2018-03-16,LAL,91.0,MIA,92.0,MIA,1,0.536585,0.625


Excellent, we can now add the away 8 game win%, home 41 game win%, and home 8 game win% to scores in the same manner. To avoid repeating code, we'll make a helper function that does just that.

In [35]:
def merge_team_col_to_scores(team_col,team_loc):
    """
    team_col = string, name of the column on team df
    team_loc = string, "home team" or "away team" (whether the col refers to home or away team)
    """
    vals = []
    for i in range(len(scores)):
        team = scores[team_loc][i]
        team_df = team_dfs[team]
        val = team_df[team_df['game_id']==i][team_col].iloc[0]
        vals.append(val)
    scores[team_loc+' '+team_col] = vals

In [36]:
merge_team_col_to_scores('8 game win%','away team')
merge_team_col_to_scores('41 game win%','home team')
merge_team_col_to_scores('8 game win%','home team')

In [37]:
scores.tail()

,game_id,date,home team,home pts,away team,away pts,winner,away team 41 game win%,away team 8 game win%,home team 41 game win%,home team 8 game win%
2273,2273,2018-03-17,WAS,109.0,IND,102.0,WAS,0.585366,0.625,0.585366,0.500
2274,2274,2018-03-18,LAC,109.0,POR,122.0,POR,0.682927,1.000,0.634146,0.500
2275,2275,2018-03-18,MIN,120.0,HOU,129.0,HOU,0.756098,0.875,0.560976,0.375
2276,2276,2018-03-18,NOP,108.0,BOS,89.0,NOP,0.585366,0.500,0.609756,0.500
2277,2277,2018-03-18,TOR,125.0,OKC,132.0,OKC,0.658537,0.750,0.756098,0.875


Success!

In [38]:
scores.to_csv('game_scores.csv')

Alright, at this point I think we have enough potentially useful features to start building our model. Continue reading in the next notebook!